# Clustering the neighborhhods in Toronto

In [1]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [2]:
import folium
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import requests
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors

## Import the neighborhood data

In [3]:
os.getcwd()

'/resources/labs/DP0701EN'

In [4]:
#Let's import the dataframe we created in the last notebook.
toronto_df = pd.read_excel('/resources/labs/DP0701EN/toronto_final.xlsx')

toronto_df.head()

,Postal codes,Boroughs,neighborhoods,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [5]:
# Let's only use the boroughs with "TORONTO" in it, as suggested.

df = toronto_df[toronto_df['Boroughs'].str.contains('Toronto')].reset_index(drop = True)

df.head()

,Postal codes,Boroughs,neighborhoods,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [6]:
#Let's get the coordinates for toronto and plot the neighborhoods on a map.

toronto_coord = Nominatim(user_agent = 'locator').geocode('toronto,Canada')

t_latitude = toronto_coord.latitude
t_longitude = toronto_coord.longitude

print("Toronto's latitude : {}\nToronto's longitude : {}".format(t_latitude,t_longitude))

Toronto's latitude : 43.6534817
Toronto's longitude : -79.3839347


In [7]:
t_map = folium.Map(location = [t_latitude,t_longitude], zoom_start = 12)

for lat,long,neighborhood in zip(df.Latitude,df.Longitude,df.neighborhoods):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,long],
    popup = label,
    radius = 5,
    color = 'red',
    fill = True,
    fill_color='#50000cc',
    fill_opacity=0.7,
    parse_html=False
    ).add_to(t_map)

t_map

The folium map might not be visible on GitHub.

In [8]:
len(set(df['Boroughs']))

4

There are 4 different boroughs with multiple neighborhoods in some boroughs.

# Define Foursquare Credentials and Version

In [9]:
CLIENT_ID = 'K4LYUFZPJ30CIEYJ2KCV5CFDAUWT0TS2U3G1ZV2TA15FO2ZG' # Foursquare ID
CLIENT_SECRET = 'DFLWK3MJTIOBR0ZDBPPPYSW544HGNGKRWOUQMD02UFLDLSG2' # Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
radius = 500

In [10]:
#Function to get the nearby trending venues from the neighborhoods using foursquare api
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [11]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [12]:
#Creating a new dataframe with the trending venues near all the neighborhoods.
toronto_venues = getNearbyVenues(names=df['neighborhoods'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )



Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town,

In [13]:
print(toronto_venues.shape)
toronto_venues.head()

(1616, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [14]:
#Checking how many venues were returned for each neighborhood

toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,56,56,56,56,56,56
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
Business reply mail Processing Centre,18,18,18,18,18,18
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,64,64,64,64,64,64
Christie,17,17,17,17,17,17
Church and Wellesley,76,76,76,76,76,76
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,36,36,36,36,36,36


Commerce Court, Victoria Hotel,First Canadian Place, Underground city,Toronto Dominion Centre, Design Exchange,Garden District, Ryerson,Harbourfront East, Union Station, Toronto Islands have the highest number of trending locations.

In [15]:
#Number of different types of venues

print("There are {} types of venues in the selected neighborhoods of Toronto".format(len(toronto_venues['Venue Category'].unique())))

There are 237 types of venues in the selected neighborhoods of Toronto


### Analyze the neighborhoods

In [16]:
#Let's one hot encode the venue categories.
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

#Adding the neighborhood details.
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']


#Reordering the columns such that neighborhood comes first.

list1 = ['Neighborhood','Afghan Restaurant', 'Airport', 'Airport Food Court',
       'Airport Gate', 'Airport Lounge', 'Airport Service',
       'Airport Terminal', 'American Restaurant', 'Antique Shop',
       'Aquarium', 'Art Gallery', 'Art Museum', 'Arts & Crafts Store',
       'Asian Restaurant', 'Athletics & Sports', 'Auto Workshop',
       'BBQ Joint', 'Baby Store', 'Bagel Shop', 'Bakery', 'Bank', 'Bar',
       'Baseball Stadium', 'Basketball Stadium', 'Beach',
       'Bed & Breakfast', 'Beer Bar', 'Beer Store', 'Belgian Restaurant',
       'Bistro', 'Boat or Ferry', 'Bookstore', 'Boutique',
       'Brazilian Restaurant', 'Breakfast Spot', 'Brewery',
       'Bubble Tea Shop', 'Building', 'Burger Joint', 'Burrito Place',
       'Bus Line', 'Business Service', 'Butcher', 'Café',
       'Cajun / Creole Restaurant', 'Camera Store', 'Candy Store',
       'Caribbean Restaurant', 'Cheese Shop', 'Chinese Restaurant',
       'Chocolate Shop', 'Church', 'Climbing Gym', 'Clothing Store',
       'Cocktail Bar', 'Coffee Shop', 'College Arts Building',
       'College Auditorium', 'College Gym', 'College Rec Center',
       'Colombian Restaurant', 'Comfort Food Restaurant', 'Comic Shop',
       'Concert Hall', 'Convenience Store', 'Convention Center',
       'Cosmetics Shop', 'Coworking Space', 'Creperie',
       'Cuban Restaurant', 'Cupcake Shop', 'Cycle Studio', 'Dance Studio',
       'Deli / Bodega', 'Department Store', 'Dessert Shop', 'Diner',
       'Discount Store', 'Distribution Center', 'Dog Run',
       'Doner Restaurant', 'Donut Shop', 'Dumpling Restaurant',
       'Eastern European Restaurant', 'Electronics Store',
       'Ethiopian Restaurant', 'Event Space', 'Falafel Restaurant',
       'Farmers Market', 'Fast Food Restaurant', 'Filipino Restaurant',
       'Fish & Chips Shop', 'Fish Market', 'Flea Market', 'Flower Shop',
       'Food', 'Food & Drink Shop', 'Food Court', 'Food Truck',
       'Fountain', 'French Restaurant', 'Fried Chicken Joint',
       'Frozen Yogurt Shop', 'Fruit & Vegetable Store',
       'Furniture / Home Store', 'Gaming Cafe', 'Garden', 'Garden Center',
       'Gas Station', 'Gastropub', 'Gay Bar', 'General Entertainment',
       'General Travel', 'German Restaurant', 'Gift Shop',
       'Gluten-free Restaurant', 'Gourmet Shop', 'Greek Restaurant',
       'Grocery Store', 'Gym', 'Gym / Fitness Center', 'Harbor / Marina',
       'Health & Beauty Service', 'Health Food Store', 'Historic Site',
       'History Museum', 'Hobby Shop', 'Hookah Bar', 'Hospital',
       'Hot Dog Joint', 'Hotel', 'Hotel Bar', 'IT Services',
       'Ice Cream Shop', 'Indian Restaurant', 'Indie Movie Theater',
       'Intersection', 'Irish Pub', 'Italian Restaurant',
       'Japanese Restaurant', 'Jazz Club', 'Jewelry Store', 'Juice Bar',
       'Korean Restaurant', 'Lake', 'Latin American Restaurant',
       'Light Rail Station', 'Lingerie Store', 'Liquor Store', 'Lounge',
       'Mac & Cheese Joint', 'Market', 'Martial Arts Dojo',
       'Mediterranean Restaurant', "Men's Store", 'Mexican Restaurant',
       'Middle Eastern Restaurant', 'Miscellaneous Shop',
       'Modern European Restaurant', 'Molecular Gastronomy Restaurant',
       'Monument / Landmark', 'Moroccan Restaurant', 'Movie Theater',
       'Museum', 'Music Venue', 'New American Restaurant',
       'Nightclub', 'Noodle House', 'Office', 'Opera House',
       'Optical Shop', 'Organic Grocery', 'Other Great Outdoors', 'Park',
       'Performing Arts Venue', 'Pet Store', 'Pharmacy', 'Pizza Place',
       'Plane', 'Playground', 'Plaza', 'Poke Place', 'Pool',
       'Poutine Place', 'Pub', 'Ramen Restaurant', 'Record Shop',
       'Recording Studio', 'Rental Car Location', 'Restaurant',
       'Roof Deck', 'Sake Bar', 'Salad Place', 'Salon / Barbershop',
       'Sandwich Place', 'Scenic Lookout', 'Sculpture Garden',
       'Seafood Restaurant', 'Shoe Store', 'Shopping Mall', 'Skate Park',
       'Skating Rink', 'Smoke Shop', 'Smoothie Shop', 'Snack Place',
       'Soup Place', 'South American Restaurant', 'Spa', 'Speakeasy',
       'Sporting Goods Shop', 'Sports Bar', 'Stadium', 'Stationery Store',
       'Steakhouse', 'Strip Club', 'Supermarket', 'Sushi Restaurant',
       'Swim School', 'Taco Place', 'Tailor Shop', 'Taiwanese Restaurant',
       'Tanning Salon', 'Tea Room', 'Thai Restaurant', 'Theater',
       'Theme Restaurant', 'Toy / Game Store', 'Trail', 'Train Station',
       'Vegetarian / Vegan Restaurant', 'Video Game Store',
       'Vietnamese Restaurant', 'Wine Bar', 'Wings Joint',
       "Women's Store", 'Yoga Studio']

toronto_onehot = toronto_onehot[list1]
toronto_onehot.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Grouping by the neighborhood and taking mean occurence of the venue type

In [17]:
toronto_onehot_grouped = toronto_onehot.groupby(['Neighborhood']).mean().reset_index()

toronto_onehot_grouped.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,...,0.0,0.0,0.0,0.017857,0.0,0.0,0.000000,0.0,0.0,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
2,Business reply mail Processing Centre,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.055556
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
4,Central Bay Street,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,...,0.0,0.0,0.0,0.015625,0.0,0.0,0.015625,0.0,0.0,0.015625


### Checking the 10 most common values for each neighborhood

In [18]:
for i,hood in enumerate(toronto_onehot_grouped['Neighborhood']):
    print('{}.'.format(i+1) + "----"+hood+"----")
    temp = toronto_onehot_grouped[toronto_onehot_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    print(temp.sort_values(by='freq',ascending = False).head(10))
    
    

1.----Berczy Park----
                  venue       freq
56          Coffee Shop  0.0714286
55         Cocktail Bar  0.0535714
44                 Café  0.0357143
49          Cheese Shop  0.0357143
198  Seafood Restaurant  0.0357143
27             Beer Bar  0.0357143
190          Restaurant  0.0357143
20               Bakery  0.0357143
119       Grocery Store  0.0178571
77                Diner  0.0178571
2.----Brockton, Parkdale Village, Exhibition Place----
                      venue       freq
44                     Café   0.136364
35           Breakfast Spot  0.0909091
56              Coffee Shop  0.0909091
119           Grocery Store  0.0454545
137            Intersection  0.0454545
40            Burrito Place  0.0454545
212                 Stadium  0.0454545
22                      Bar  0.0454545
105  Furniture / Home Store  0.0454545
20                   Bakery  0.0454545
3.----Business reply mail Processing Centre----
                  venue       freq
147  Light Rail Station   

## Let's put that into a pandas dataframe
First, let's write a function to sort the venues in descending order.

In [19]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Let's create a new dataframe with top 10 venues

In [20]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_onehot_grouped['Neighborhood']

for ind in np.arange(toronto_onehot_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_onehot_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Café,Beer Bar,Bakery,Seafood Restaurant,Cheese Shop,Restaurant,Pub,Farmers Market
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Burrito Place,Italian Restaurant,Restaurant,Intersection,Stadium,Bar,Bakery
2,Business reply mail Processing Centre,Light Rail Station,Auto Workshop,Skate Park,Brewery,Burrito Place,Spa,Farmers Market,Fast Food Restaurant,Restaurant,Recording Studio
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Sculpture Garden,Harbor / Marina,Rental Car Location,Plane,Coffee Shop,Boat or Ferry,Bar
4,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Japanese Restaurant,Ice Cream Shop,Salad Place,Bubble Tea Shop,Thai Restaurant,Burger Joint


Although this dataframe will not be used for clustering, It provides a very good idea of the most trending venues in all the neighborhoods in toronto

# Clustering the Neighborhoods

Let's group the neighborhoods into 5 clusters

In [21]:
kclusters = 5

toronto_grouped_clustering = toronto_onehot_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [22]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='neighborhoods')

toronto_merged.head() # check the last columns!

,Postal codes,Boroughs,neighborhoods,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Theater,Mexican Restaurant,Shoe Store,Restaurant
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Yoga Studio,College Auditorium,Beer Bar,Smoothie Shop,Sandwich Place,Restaurant,Burrito Place,Café,Creperie
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Clothing Store,Coffee Shop,Restaurant,Café,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Bubble Tea Shop,Middle Eastern Restaurant,Bookstore
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Café,Coffee Shop,Cocktail Bar,Gastropub,American Restaurant,Clothing Store,Italian Restaurant,Restaurant,Beer Bar,Moroccan Restaurant
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Trail,Pub,Health Food Store,Dog Run,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Yoga Studio


In [23]:
#Visualizing the clusters

latitude = t_latitude
longitude = t_longitude

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['neighborhoods'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine the clusters

### Cluster 1

In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + [2] + list(range(5, toronto_merged.shape[1]))]]

,Boroughs,neighborhoods,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,East Toronto,The Beaches,0,Trail,Pub,Health Food Store,Dog Run,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Yoga Studio


##### Cluster 1 is situated near the beach and the people go out on trails
##### Cluster can be classified as beach goers

### Cluster 2

In [38]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + [2] + list(range(5, toronto_merged.shape[1]))]]

,Boroughs,neighborhoods,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,"Regent Park, Harbourfront",1,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Theater,Mexican Restaurant,Shoe Store,Restaurant
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",1,Coffee Shop,Yoga Studio,College Auditorium,Beer Bar,Smoothie Shop,Sandwich Place,Restaurant,Burrito Place,Café,Creperie
2,Downtown Toronto,"Garden District, Ryerson",1,Clothing Store,Coffee Shop,Restaurant,Café,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Bubble Tea Shop,Middle Eastern Restaurant,Bookstore
3,Downtown Toronto,St. James Town,1,Café,Coffee Shop,Cocktail Bar,Gastropub,American Restaurant,Clothing Store,Italian Restaurant,Restaurant,Beer Bar,Moroccan Restaurant
5,Downtown Toronto,Berczy Park,1,Coffee Shop,Cocktail Bar,Café,Beer Bar,Bakery,Seafood Restaurant,Cheese Shop,Restaurant,Pub,Farmers Market
6,Downtown Toronto,Central Bay Street,1,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Japanese Restaurant,Ice Cream Shop,Salad Place,Bubble Tea Shop,Thai Restaurant,Burger Joint
7,Downtown Toronto,Christie,1,Grocery Store,Café,Park,Restaurant,Candy Store,Athletics & Sports,Diner,Italian Restaurant,Baby Store,Nightclub
8,Downtown Toronto,"Richmond, Adelaide, King",1,Coffee Shop,Café,Restaurant,Deli / Bodega,Thai Restaurant,Hotel,Clothing Store,Gym,Bookstore,Salad Place
9,West Toronto,"Dufferin, Dovercourt Village",1,Bakery,Pharmacy,Brewery,Park,Grocery Store,Supermarket,Bar,Coffee Shop,Music Venue,Middle Eastern Restaurant
10,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",1,Coffee Shop,Aquarium,Hotel,Café,Fried Chicken Joint,Sporting Goods Shop,Restaurant,Italian Restaurant,Scenic Lookout,Brewery


##### Cluster 1 is in the heart of the city, i.e the downtown Toronto. People here are mostly interested in Coffee shops and Bistros
##### Cluster can be classified as downtown City folks

### Cluster 3

In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + [2] + list(range(5, toronto_merged.shape[1]))]]

,Boroughs,neighborhoods,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,Lawrence Park,2,Park,Bus Line,Swim School,Event Space,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
21,Central Toronto,Forest Hill North & West,2,Park,Jewelry Store,Trail,Bus Line,Sushi Restaurant,Yoga Studio,Department Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
33,Downtown Toronto,Rosedale,2,Park,Playground,Trail,Cycle Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


##### This cluster has a lot of parks,grounds and even spaces in general
##### The cluster can be classified as Outdoorsy folks

### Cluster 4

In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + [2] + list(range(5, toronto_merged.shape[1]))]]

,Boroughs,neighborhoods,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,Central Toronto,"Moore Park, Summerhill East",3,Playground,Trail,Intersection,Yoga Studio,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


##### This cluster has a lot of activity oriented people with playground,trail,Yoga studio,intersection etc

##### The cluster can be classified as Fitness oriented folks

### Cluster 5

In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + [2] + list(range(5, toronto_merged.shape[1]))]]

,Boroughs,neighborhoods,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,Roselawn,4,Garden,Music Venue,Health & Beauty Service,Ice Cream Shop,Yoga Studio,Department Store,Dessert Shop,Diner,Discount Store,Dog Run


##### This cluster has gardens as the most trending venue.
##### The cluster can be classified as Nature lovers